## TODO:
- Fix BPM feature

# Creating the proper techno dataset

Assuming we have downloaded the spotify playlists for Proper and NonProper Techno using deemix by pasting the spotify playlist URL in deemix, in this notebook we organize the audio files by class into a pandas dataframe. We are going to download the proper techno spotify playlist and use that as the proper techno true labels. For the negative classes we will take some hard techno, trance, melodic techno spotify playlists as the negatives


In [1]:
import os
!pwd
os.chdir("..") # need to cd into the directory first..
os.chdir("data") # need to cd into the directory first..
!pwd

/Users/zein/proper-classifier/src
/Users/zein/proper-classifier/data


In [ ]:
import os
os.chdir("proper-classifier/data/") # need to cd into the directory first..


In [2]:
import pandas as pd
import os
import librosa
import librosa.display
import numpy as np
import IPython.display as ipd
import matplotlib.pyplot as plt
%matplotlib inline


## EDA

In [ ]:
# testing librosa library on one audio file, setting the params for reading each audio file
# os.chdir("ProperTechno/")
# !pwd
file_name = 'Volster - EXPOSITION F.mp3'
audio_data, sampling_rate = librosa.load(file_name, mono=True) # Mono param here
librosa.display.waveshow(audio_data, sr=sampling_rate)
ipd.Audio(file_name)

In [ ]:
audio_data

In [ ]:
mfccs = librosa.feature.mfcc(y=audio_data, sr=sampling_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs.T,axis=0)

In [ ]:
mfccs_scaled_features

<p> Issue: dealing with Mono is more straightforward but stereo provides more information. The problem is when we do stereo it is not immediately clear how we process the features because we have one set of features for each channel. Do we combine the values together into a 1d array by adding the features in this order ch1,ch2,ch1,ch2? Potentially we could try an initial run and training of the model in mono then try doing it in stereo and see if it makes a difference.<p> 

## Extract the features over each file and create pandas dataframe with: filename (id), features, label

In [ ]:
# Deprecate
def features_extractor(file):
    '''
    Function that takes as input an audio file and extracts the mfccs features + scales the values
    Use in a for loop to extract and scale the features for all tracks in the dataset
    '''
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast', mono=False) 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40, )
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    return mfccs_scaled_features

# Loop through the ProperTechno folder and the nonpropertechno folder and extract the features for all tracks
# Add them to the pandas data frame with [track title, features, label]
# can do it sequentially starting with ProperTechno tracks, add to DF, then repeat for nonpropertechno and add to df

#Code from the tutorial, needs to be adapted to my custom dataset.
extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    final_class_labels=row["class"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

    
extracted_features_df=pd.DataFrame(extracted_features,columns=['track','feature','class'])
extracted_features_df.head(10)

# Getting the features

We are computing 40 MFCCs, the tempo and beat features of the track, and the Spectral Centroid and Bandwith using librosa.

We extract the mean spectral centroid, mean spectral bandwidth, and mean zero crossing rate with np.mean() to get a single value for each feature per track. This is a common approach for summarizing frame-level features across time.

## Dev notes

Skipping tempo calculation using librosa as is not correct due to the algorithm not being optimized for electronic music. Rekordbox's algorithm is much better suited for our case. Not available through an API though, so need to use this library https://github.com/dylanljones/pyrekordbox

In [11]:
import os
import librosa
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Set the directory containing the audio files
audio_dir = "ProperTechno/"

# Initialize an empty list to store the data
data = []

# Loop through each file in the directory
for subdir, dirs, files in os.walk(audio_dir):
    for file in files:
        # Load the audio file
        file_path = os.path.join(subdir, file)
        print(file)
        y, sr = librosa.load(file_path, mono=True, duration=30)
        
        parent_dir = os.path.basename(os.path.dirname(file_path))

        # Extract the MFCC, spectral centroid and bandwidth, beat features, and zero crossing rate features
        
        #MFCCs features - Decrease the n_mfcc param?
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20, hop_length=512)
        
#         print("MFCCs Shape: {}".format(mfccs.shape))
        
        # Spectral features
        spec_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        spec_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
        
        # Beat features
#         full_track, full_sr = librosa.load(file_path, mono=True, duration=30)
#         tempo, beats = librosa.beat.beat_track(y=full_track, sr=full_sr) #tempo returns BPM of the track    
        tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
        beat_times = librosa.frames_to_time(beats, sr=sr)
        ibi = np.diff(beat_times)
        ibi_mean = np.mean(ibi)
        ibi_min = np.min(ibi)
        ibi_max = np.max(ibi)
        
        # Zero Crossing Rate feature
        zcr = np.mean(librosa.feature.zero_crossing_rate(y=y))
        
        # Create a row of data with the filename, MFCCs, spectral centroid, spectral bandwidth, and beat features
        row = [file, parent_dir]
        
#         print("To list shape: {}".format(np.array(mfccs.tolist()).shape))
#         print("Flattened to list shape: {}".format(np.array(mfccs.flatten().tolist()).shape))

        row.extend(mfccs.flatten().tolist())
        
        row.append(spec_centroid)
        row.append(spec_bandwidth)
#         row.append(tempo)
        row.append(ibi_mean)
        row.append(ibi_min)
        row.append(ibi_max)
        row.append(zcr)

        # Append the row to the data list
        data.append(row)

# Create a dataframe from the data list
columns = ["track", "label"]
for i in range(25840):
    columns.append("mfccs_feature_" + str(i+1))
columns.append("spec_centroid")
columns.append("spec_bandwidth")
# columns.append("tempo")
columns.append("ibi_mean")
columns.append("ibi_min")
columns.append("ibi_max")
columns.append("zcr")

df = pd.DataFrame(data, columns=columns)
df

Frank Biazzi - a1. Introspection [www.slider.kz].mp3
Volster - EXPOSITION F.mp3
Hitam - Uxuxhumana.mp3
Sven Dedek - Black.mp3
Fireground - IT055 - Recreation - 01 Recreation.mp3
Arthur Robert - Forceful.mp3
BLANKA - Transversal -SK11X017- - 01 La Buena Suerte.mp3
Edit Select - RNGD - Lanna.mp3
Beau Didier - Get Up! [BEAU006].mp3
Felix Leifur - Brot 12.mp3
Altinbas - Illumination.mp3
Beau Didier - Funky Feeling [BEAU006].mp3
Adam Beyer - Patches [www.slider.kz].mp3


,track,label,mfccs_feature_1,mfccs_feature_2,mfccs_feature_3,mfccs_feature_4,mfccs_feature_5,mfccs_feature_6,mfccs_feature_7,mfccs_feature_8,...,mfccs_feature_25837,mfccs_feature_25838,mfccs_feature_25839,mfccs_feature_25840,spec_centroid,spec_bandwidth,ibi_mean,ibi_min,ibi_max,zcr
0,Frank Biazzi - a1. Introspection [www.slider.k...,ProperTechno,-20.249231,-14.057375,-104.332077,-159.476501,-193.839569,-145.227570,-86.645287,-92.865059,...,-0.036119,8.360538,10.199444,5.835332,4248.796238,3064.738383,0.438060,0.417959,0.441179,0.278671
1,Volster - EXPOSITION F.mp3,ProperTechno,-471.156189,-471.156189,-471.156189,-471.156189,-471.156189,-471.156189,-469.059174,-443.035919,...,3.697540,7.360375,6.293511,2.934453,1141.623166,1930.930012,0.648095,0.626939,0.673379,0.017417
2,Hitam - Uxuxhumana.mp3,ProperTechno,-458.978577,-458.978577,-458.978577,-458.978577,-458.978577,-458.978577,-458.978577,-458.978577,...,-1.793820,-2.254179,-1.118765,3.056025,418.264564,843.031562,0.434846,0.417959,0.441179,0.009407
3,Sven Dedek - Black.mp3,ProperTechno,-37.479687,22.478897,-39.087749,-159.876862,-201.036575,-201.511551,-185.444855,-173.454636,...,3.295895,0.203048,0.365886,1.840741,1896.461274,2501.234849,0.543697,0.510839,0.650159,0.041583
4,Fireground - IT055 - Recreation - 01 Recreatio...,ProperTechno,-152.982117,-128.793732,-216.944687,-306.299164,-340.194000,-354.286011,-354.318420,-351.337494,...,6.582436,7.807012,4.793015,3.620410,2314.942662,2367.024136,0.413979,0.394739,0.417959,0.081181
5,Arthur Robert - Forceful.mp3,ProperTechno,-37.355930,-57.164169,-186.223679,-263.562225,-167.732834,-93.397133,-98.538605,-148.619385,...,0.730058,-3.057313,-3.411319,-0.859603,2504.870629,3133.456931,0.441536,0.417959,0.557279,0.036756
6,BLANKA - Transversal -SK11X017- - 01 La Buena ...,ProperTechno,-484.146698,-484.146698,-484.146698,-484.146698,-484.146698,-484.146698,-484.146698,-424.783875,...,-0.539492,4.048290,-8.931528,-4.091005,950.821846,1443.440732,0.850695,0.673379,0.859138,0.024705
7,Edit Select - RNGD - Lanna.mp3,ProperTechno,-103.553680,-107.033669,-193.271866,-245.262054,-201.491760,-123.562233,-129.007599,-186.690521,...,-1.422752,0.783077,-0.762225,2.641242,732.008091,1328.740569,0.539316,0.441179,0.626939,0.018890
8,Beau Didier - Get Up! [BEAU006].mp3,ProperTechno,43.165749,27.309042,-86.475533,-149.678268,-143.784622,-146.747360,-184.583420,-207.793320,...,0.859228,-1.400749,-2.314552,-2.648981,3832.838469,2735.528681,0.422334,0.417959,0.441179,0.243101
9,Felix Leifur - Brot 12.mp3,ProperTechno,-83.217155,-40.215252,-84.610489,-143.799820,-179.783707,-208.535019,-221.713928,-225.449127,...,3.314949,5.141429,6.468519,6.105281,4309.365528,2781.735009,0.857687,0.835918,0.882358,0.329726


In [ ]:
# Save the dataframe to a CSV file
df.to_csv("df_track_features.csv", index=False)

In [ ]:
df = pd.read_csv('df_track_features.csv')
df

## Extract the BPM from Rekorbox

To extract the BPM value that Rekordbox calculated, we first need to download the playlist locally, then import it in a rekordbox playlist.

In [ ]:
import pyrekordbox
import json

# Connect to Rekordbox
rekordbox = pyrekordbox.Rekordbox()

# Set the name of the playlist to analyze
playlist_name = "My Playlist"

# Get the tracks in the playlist
playlist_tracks = rekordbox.get_playlist_tracks(playlist_name)

# Loop through the tracks and get the BPM
for track in playlist_tracks:
    # Get the track info
    track_info = rekordbox.get_track_info(track['id'])
    
    # Get the BPM from the track info
    bpm = track_info.get('TEMPO', {}).get('value')
    
    # Print the track name and BPM
    print(f"{track['name']} - BPM: {bpm}")

# Disconnect from Rekordbox
rekordbox.disconnect()


# Preprocessing (Dim reduct, and standardization)

We want to scale the features in the df appropriately here to normalize everything.

In [ ]:
#standardize all the features


# Perform PCA on the standardized MFCC features
pca = PCA(n_components=40)
principal_components = pca.fit_transform(mfcc_features)

# Add the principal components back to the dataframe
for i in range(num_components):
    data[f'PC{i+1}'] = principal_components[:,i]